# 🏗️ ETAPA 1: FUNDAÇÃO E PREPARAÇÃO

## 🎯 **O que esta etapa faz**
Prepara todo o ambiente necessário para executar o pipeline NIC ETL com segurança e eficiência.

## 🤔 **Por que esta etapa é necessária**
Antes de processar documentos, precisamos:
- ✅ Verificar se todas as credenciais estão corretas
- ✅ Validar conectividade com serviços externos (GitLab, Qdrant)
- ✅ Configurar parâmetros do pipeline
- ✅ Garantir que o ambiente está pronto

## ⚙️ **Como funciona**
1. **Carrega configurações** do arquivo `.env`
2. **Valida credenciais** do GitLab e Qdrant
3. **Testa conectividade** com os serviços
4. **Define parâmetros** do processamento
5. **Salva configuração validada** para próximas etapas

## 📊 **Parâmetros que você pode ajustar**
- `gitlab_url`: URL do GitLab (padrão: http://gitlab.processa.info)
- `target_folder`: Pasta dos documentos (padrão: 30-Aprovados)
- `file_extensions`: Tipos de arquivo aceitos (padrão: .pdf, .docx, .md, .txt, .png, .jpg, .jpeg)
- `chunk_size`: Tamanho dos chunks (padrão: 500 tokens)
- `embedding_model`: Modelo de embeddings (padrão: BAAI/bge-m3)

## 👁️ **O que esperar de saída**
- ✅ Configuração validada e salva
- ✅ Credenciais testadas
- ✅ Ambiente preparado para próximas etapas
- ✅ Arquivo `stage_01_foundation.json` criado

## ⚠️ **Pontos importantes**
- Esta etapa **DEVE ser executada primeiro**
- Se falhar, **verifique suas credenciais** no arquivo `.env`
- **Modo independente** disponível para testes

## 🔧 Configuração e Imports

In [1]:
# Imports essenciais
import sys
import os
from pathlib import Path
from datetime import datetime
import json
from dotenv import load_dotenv

# Adicionar biblioteca simplificada ao path
sys.path.insert(0, str(Path().parent / "src"))

# Imports da biblioteca NIC ETL
from pipeline_utils import (
    PipelineState, 
    check_prerequisites, 
    show_pipeline_progress,
    get_timestamp
)
from gitlab_utils import validate_gitlab_connection
from qdrant_utils import validate_qdrant_connection

print("🏗️ ETAPA 1: FUNDAÇÃO E PREPARAÇÃO")
print("=" * 50)
print(f"📁 Diretório: {Path.cwd()}")
print(f"🕒 Iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✅ Imports carregados com sucesso")

🏗️ ETAPA 1: FUNDAÇÃO E PREPARAÇÃO
📁 Diretório: /home/coder/nic-lab/nic-etl/notebooks
🕒 Iniciado: 2025-08-17 18:27:08

✅ Imports carregados com sucesso


## 📊 Status do Pipeline

In [2]:
# 📊 VERIFICAÇÃO DE STATUS DO PIPELINE
print("📊 Verificando status do pipeline...")

# Verificar se esta etapa já foi executada
state = PipelineState()
if state.is_stage_completed(1):
    print("ℹ️  Esta etapa já foi concluída anteriormente")
    print("📋 Para reexecutar, use reset_pipeline() no master notebook")

print("✅ Pronto para executar etapa de fundação")
print("\n" + "=" * 50)

📊 Verificando status do pipeline...
ℹ️  Esta etapa já foi concluída anteriormente
📋 Para reexecutar, use reset_pipeline() no master notebook
✅ Pronto para executar etapa de fundação



## 📋 Carregamento de Configurações

In [3]:
# 📋 CARREGAMENTO DE CONFIGURAÇÕES
print("📋 Carregando configurações do ambiente...")

# Determinar ambiente
ENVIRONMENT = os.getenv("ENVIRONMENT", "development")
print(f"🌍 Ambiente: {ENVIRONMENT}")

# Carregar arquivo de configuração
env_files = [f".env.{ENVIRONMENT}", ".env"]
env_loaded = False

for env_file in env_files:
    env_path = Path().parent / env_file
    if env_path.exists():
        print(f"📄 Carregando: {env_file}")
        load_dotenv(env_path, override=True)
        env_loaded = True
        break

if not env_loaded:
    print("⚠️  Nenhum arquivo .env encontrado - usando variáveis do sistema")

# 🔧 CONFIGURAÇÕES PRINCIPAIS
config = {
    "environment": ENVIRONMENT,
    "timestamp": get_timestamp(),
    
    # GitLab
    "gitlab": {
        "url": os.getenv("GITLAB_URL", "http://gitlab.processa.info"),
        "repository": os.getenv("GITLAB_REPOSITORY", "nic/documentacao/base-de-conhecimento"),
        "branch": os.getenv("GITLAB_BRANCH", "main"),
        "target_folder": os.getenv("GITLAB_TARGET_FOLDER", "30-Aprovados"),
        "token": os.getenv("GITLAB_ACCESS_TOKEN", "")
    },
    
    # Qdrant
    "qdrant": {
        "url": os.getenv("QDRANT_URL", "https://qdrant.codrstudio.dev/"),
        "collection_name": os.getenv("QDRANT_COLLECTION", f"nic_{ENVIRONMENT}"),
        "api_key": os.getenv("QDRANT_API_KEY", "")
    },
    
    # Processamento
    "processing": {
        "file_extensions": [".pdf", ".docx", ".md", ".txt", ".png", ".jpg", ".jpeg"],
        "max_file_size_mb": int(os.getenv("MAX_FILE_SIZE_MB", "50")),
        "chunk_size": int(os.getenv("CHUNK_SIZE", "500")),
        "chunk_overlap": int(os.getenv("CHUNK_OVERLAP", "100")),
        "embedding_model": os.getenv("EMBEDDING_MODEL", "BAAI/bge-m3"),
        "batch_size": int(os.getenv("BATCH_SIZE", "32"))
    },
    
    # Pipeline
    "pipeline": {
        "max_concurrent_docs": int(os.getenv("MAX_CONCURRENT_DOCS", "4")),
        "enable_cache": os.getenv("ENABLE_CACHE", "true").lower() == "true",
        "enable_monitoring": os.getenv("ENABLE_MONITORING", "true").lower() == "true"
    }
}

print("\n📊 Configurações carregadas:")
print(f"   🔗 GitLab: {config['gitlab']['url']}")
print(f"   📂 Repositório: {config['gitlab']['repository']}")
print(f"   📁 Pasta: {config['gitlab']['target_folder']}")
print(f"   🧠 Qdrant: {config['qdrant']['url']}")
print(f"   📊 Collection: {config['qdrant']['collection_name']}")
print(f"   🤖 Modelo: {config['processing']['embedding_model']}")
print(f"   📏 Chunk size: {config['processing']['chunk_size']} tokens")

print("\n✅ Configurações carregadas com sucesso")

📋 Carregando configurações do ambiente...
🌍 Ambiente: development
⚠️  Nenhum arquivo .env encontrado - usando variáveis do sistema

📊 Configurações carregadas:
   🔗 GitLab: http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
   📂 Repositório: nic/documentacao/base-de-conhecimento
   📁 Pasta: 30-Aprovados
   🧠 Qdrant: http://qdrant.codrstudio.dev:6333
   📊 Collection: nic
   🤖 Modelo: BAAI/bge-m3
   📏 Chunk size: 500 tokens

✅ Configurações carregadas com sucesso


## 🔐 Validação de Credenciais

In [4]:
# 🔐 VALIDAÇÃO DE CREDENCIAIS
print("🔐 Validando credenciais e conectividade...")

validation_results = {
    "gitlab_valid": False,
    "qdrant_valid": False,
    "config_complete": False,
    "errors": []
}

# 📡 Validar GitLab
print("\n📡 Testando conexão GitLab...")
try:
    if config["gitlab"]["token"]:
        gitlab_valid = validate_gitlab_connection(
            config["gitlab"]["url"], 
            config["gitlab"]["token"]
        )
        validation_results["gitlab_valid"] = gitlab_valid
        if gitlab_valid:
            print("✅ GitLab: Conexão validada")
        else:
            print("❌ GitLab: Falha na validação")
            validation_results["errors"].append("GitLab connection failed")
    else:
        print("❌ GitLab: Token não configurado (GITLAB_ACCESS_TOKEN)")
        validation_results["errors"].append("GitLab token missing")
        # Token é obrigatório para execução
        if not config["gitlab"]["token"]:
            raise ValueError("GITLAB_ACCESS_TOKEN é obrigatório")
        
except Exception as e:
    print(f"❌ GitLab: Erro na validação - {e}")
    validation_results["errors"].append(f"GitLab validation error: {e}")

# 🧠 Validar Qdrant
print("\n🧠 Testando conexão Qdrant...")
try:
    if config["qdrant"]["api_key"]:
        qdrant_valid = validate_qdrant_connection(
            config["qdrant"]["url"], 
            config["qdrant"]["api_key"]
        )
        validation_results["qdrant_valid"] = qdrant_valid
        if qdrant_valid:
            print("✅ Qdrant: Conexão validada")
        else:
            print("❌ Qdrant: Falha na validação")
            validation_results["errors"].append("Qdrant connection failed")
    else:
        print("❌ Qdrant: API Key não configurada (QDRANT_API_KEY)")
        validation_results["errors"].append("Qdrant API key missing")
        # API Key é obrigatória para execução
        if not config["qdrant"]["api_key"]:
            raise ValueError("QDRANT_API_KEY é obrigatória")
        
except Exception as e:
    print(f"❌ Qdrant: Erro na validação - {e}")
    validation_results["errors"].append(f"Qdrant validation error: {e}")

# 📋 Validar completude da configuração
required_fields = [
    config["gitlab"]["url"],
    config["gitlab"]["repository"],
    config["qdrant"]["url"],
    config["processing"]["embedding_model"]
]

if all(required_fields):
    validation_results["config_complete"] = True
    print("\n✅ Configuração: Todos os campos obrigatórios preenchidos")
else:
    print("\n❌ Configuração: Campos obrigatórios faltando")
    validation_results["errors"].append("Required configuration fields missing")

# 📊 Resumo da validação
print("\n" + "=" * 50)
print("📊 RESUMO DA VALIDAÇÃO:")
print(f"   📡 GitLab: {'✅' if validation_results['gitlab_valid'] else '❌'}")
print(f"   🧠 Qdrant: {'✅' if validation_results['qdrant_valid'] else '❌'}")
print(f"   📋 Config: {'✅' if validation_results['config_complete'] else '❌'}")

if validation_results["errors"]:
    print(f"\n⚠️  Erros encontrados ({len(validation_results['errors'])}):") 
    for i, error in enumerate(validation_results["errors"], 1):
        print(f"   {i}. {error}")

# Adicionar resultados da validação à configuração
config["validation"] = validation_results
config["validation"]["timestamp"] = get_timestamp()

🔐 Validando credenciais e conectividade...

📡 Testando conexão GitLab...
🔍 Validando conexão com http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
🔗 URL original: http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
🔗 URL da instância extraída: http://gitlab.processa.info


✅ Conexão validada para usuário: project_266_bot_91acd48207a118e1d652420e644a1f9d
✅ GitLab: Conexão validada

🧠 Testando conexão Qdrant...
🔍 Validando conexão com Qdrant: http://qdrant.codrstudio.dev:6333
🔗 Protocolo detectado: HTTP
✅ Conexão validada (HTTP) - 2 collections encontradas
✅ Qdrant: Conexão validada

✅ Configuração: Todos os campos obrigatórios preenchidos

📊 RESUMO DA VALIDAÇÃO:
   📡 GitLab: ✅
   🧠 Qdrant: ✅
   📋 Config: ✅


/home/coder/nic-lab/nic-etl/notebooks/src/qdrant_utils.py:142: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


## 📊 Preparação do Ambiente

In [5]:
# 📊 PREPARAÇÃO DO AMBIENTE
print("📊 Preparando ambiente do pipeline...")

# Inicializar state manager
state = PipelineState()

# Verificar e criar diretórios necessários
directories = [
    "documents",
    "processed", 
    "chunks",
    "embeddings",
    "metadata",
    "checkpoints"
]

print("\n📁 Verificando estrutura de diretórios:")
for directory in directories:
    dir_path = state.base_dir / directory
    dir_path.mkdir(parents=True, exist_ok=True)
    
    # Verificar se foi criado
    if dir_path.exists():
        print(f"   ✅ {directory}/")
    else:
        print(f"   ❌ {directory}/ (falha ao criar)")

# Verificar espaço em disco
import shutil
total, used, free = shutil.disk_usage(state.base_dir)
free_gb = free // (1024**3)

print(f"\n💾 Espaço em disco disponível: {free_gb} GB")
if free_gb < 5:
    print("⚠️  Aviso: Pouco espaço em disco (< 5GB)")
    config["warnings"] = config.get("warnings", [])
    config["warnings"].append("Low disk space")
else:
    print("✅ Espaço em disco adequado")

# Adicionar informações do ambiente
config["environment_info"] = {
    "pipeline_data_dir": str(state.base_dir),
    "free_disk_space_gb": free_gb,
    "python_version": sys.version.split()[0],
    "setup_timestamp": get_timestamp()
}

print("\n✅ Ambiente preparado com sucesso")

📊 Preparando ambiente do pipeline...

📁 Verificando estrutura de diretórios:
   ✅ documents/
   ✅ processed/
   ✅ chunks/
   ✅ embeddings/
   ✅ metadata/
   ✅ checkpoints/

💾 Espaço em disco disponível: 207 GB
✅ Espaço em disco adequado

✅ Ambiente preparado com sucesso


## 💾 Salvamento da Configuração

In [6]:
# 💾 SALVAMENTO DA CONFIGURAÇÃO VALIDADA
print("💾 Salvando configuração validada...")

# Preparar configuração final (sem credenciais sensíveis)
config_to_save = config.copy()

# Remover credenciais sensíveis da versão salva
config_to_save["gitlab"]["token"] = "[HIDDEN]" if config["gitlab"]["token"] else ""
config_to_save["qdrant"]["api_key"] = "[HIDDEN]" if config["qdrant"]["api_key"] else ""

# Adicionar metadados de conclusão
config_to_save["stage_completion"] = {
    "stage_number": 1,
    "stage_name": "foundation",
    "completed_at": get_timestamp(),
    "status": "success" if all([
        validation_results["config_complete"],
        validation_results["gitlab_valid"],
        validation_results["qdrant_valid"]
    ]) else "completed_with_warnings"
}

# Salvar usando PipelineState
state.save_stage_data(1, config_to_save)

# Marcar etapa como concluída
state.mark_stage_completed(1)

print(f"✅ Configuração salva: {state.metadata_dir / 'stage_01_foundation.json'}")
print(f"✅ Checkpoint criado: {state.checkpoints_dir / 'stage_01_completed.lock'}")

# Exibir resumo final
print("\n" + "=" * 60)
print("🎉 ETAPA 1 CONCLUÍDA COM SUCESSO!")
print("=" * 60)

print(f"📊 Resultados da Etapa 1:")
print(f"   🌍 Ambiente: {config['environment']}")
print(f"   📡 GitLab: {'✅ Configurado' if validation_results['gitlab_valid'] else '⚠️  Necessita atenção'}")
print(f"   🧠 Qdrant: {'✅ Configurado' if validation_results['qdrant_valid'] else '⚠️  Necessita atenção'}")
print(f"   📁 Diretórios: ✅ Criados")
print(f"   💾 Espaço: {free_gb} GB disponível")

if validation_results["errors"]:
    print(f"\n⚠️  Avisos: {len(validation_results['errors'])} itens precisam de atenção")
    print("📋 Verifique as credenciais no arquivo .env antes de continuar")
else:
    print(f"\n🚀 Pronto para próxima etapa: 02_COLETA_GITLAB.ipynb")

# Exibir progresso do pipeline
print("\n📈 Progresso do Pipeline:")
show_pipeline_progress()

💾 Salvando configuração validada...
✅ Configuração salva: pipeline_data/metadata/stage_01_foundation.json
✅ Checkpoint criado: pipeline_data/checkpoints/stage_01_completed.lock

🎉 ETAPA 1 CONCLUÍDA COM SUCESSO!
📊 Resultados da Etapa 1:
   🌍 Ambiente: development
   📡 GitLab: ✅ Configurado
   🧠 Qdrant: ✅ Configurado
   📁 Diretórios: ✅ Criados
   💾 Espaço: 207 GB disponível

🚀 Pronto para próxima etapa: 02_COLETA_GITLAB.ipynb

📈 Progresso do Pipeline:
🎯 PROGRESSO DO PIPELINE NIC ETL
════════════════════════════════════════════════════════════
✅ 01. 🏗️ Fundação e Preparação
⏳ 02. 📥 Coleta GitLab
⏳ 03. ⚙️ Processamento Docling
⏳ 04. 🔪 Segmentação em Chunks
⏳ 05. 🧠 Geração de Embeddings
⏳ 06. 💾 Armazenamento Qdrant
⏳ 07. 📊 Validação e Resultados
════════════════════════════════════════════════════════════


## 🔍 Informações de Debug (Opcional)

In [7]:
# 🔍 INFORMAÇÕES DETALHADAS PARA DEBUG
# Execute esta célula se precisar de mais informações

def show_detailed_config():
    """Mostra configuração detalhada (sem credenciais)"""
    
    print("🔍 CONFIGURAÇÃO DETALHADA")
    print("=" * 40)
    
    config_display = config.copy()
    
    # Ocultar credenciais
    if config_display["gitlab"]["token"]:
        config_display["gitlab"]["token"] = f"***{config['gitlab']['token'][-4:]}" if len(config['gitlab']['token']) > 4 else "***"
    
    if config_display["qdrant"]["api_key"]:
        config_display["qdrant"]["api_key"] = f"***{config['qdrant']['api_key'][-4:]}" if len(config['qdrant']['api_key']) > 4 else "***"
    
    # Mostrar como JSON formatado
    import json
    print(json.dumps(config_display, indent=2, ensure_ascii=False, default=str))

def test_pipeline_state():
    """Testa funções do pipeline state"""
    
    print("🧪 TESTANDO PIPELINE STATE")
    print("=" * 30)
    
    state = PipelineState()
    
    # Testar verificação de prerequisitos
    print(f"✅ Etapa 1 concluída: {state.is_stage_completed(1)}")
    print(f"⏳ Etapa 2 concluída: {state.is_stage_completed(2)}")
    
    # Testar carregamento de dados
    try:
        saved_config = state.load_stage_data(1)
        print(f"✅ Configuração salva carregada: {saved_config['stage_completion']['status']}")
    except Exception as e:
        print(f"❌ Erro ao carregar configuração: {e}")

# Funções de debug disponíveis
print("🛠️ FUNÇÕES DE DEBUG DISPONÍVEIS:")
print("   show_detailed_config() - Mostra configuração detalhada")
print("   test_pipeline_state() - Testa funções do pipeline state")
print("\n💡 Execute as funções acima se precisar de mais informações")

🛠️ FUNÇÕES DE DEBUG DISPONÍVEIS:
   show_detailed_config() - Mostra configuração detalhada
   test_pipeline_state() - Testa funções do pipeline state

💡 Execute as funções acima se precisar de mais informações


---

## ✅ **Etapa 1 Concluída!**

### 🎯 **O que foi feito:**
- ✅ Configurações carregadas e validadas
- ✅ Credenciais testadas (GitLab e Qdrant)
- ✅ Ambiente preparado (diretórios criados)
- ✅ Configuração salva para próximas etapas
- ✅ Sistema pronto para processamento

### 🚀 **Próxima etapa:**
**`02_COLETA_GITLAB.ipynb`** - Download de documentos do GitLab

### 📊 **Arquivos gerados:**
- `pipeline_data/metadata/stage_01_foundation.json` - Configuração validada
- `pipeline_data/checkpoints/stage_01_completed.lock` - Checkpoint de conclusão

### 🔧 **Para executar próxima etapa:**
1. Abra o notebook `02_COLETA_GITLAB.ipynb`
2. Execute as células em sequência
3. Ou use o `00_PIPELINE_MASTER.ipynb` para execução automática

---

**🏗️ Fundação sólida construída! Pipeline pronto para processamento de documentos.**